In [1]:
import json
import torch
import transformers
from tqdm import tqdm
from instruct_tri2tri.tinyllama_ft.conversation import conv_llama_2
def get_input_prompt(prompt):
    conv = conv_llama_2.copy()
    roles = conv_llama_2.roles
    conv.append_message(conv_llama_2.roles[0], prompt)
    conv.append_message(conv_llama_2.roles[1], None)
    prompt = conv.get_prompt()
    return prompt
model_name_or_path = 'checkpoints/tinyllama_caption2instruct_ft'
model = transformers.LlamaForCausalLM.from_pretrained(
        model_name_or_path,
        attn_implementation='flash_attention_2',
        torch_dtype=torch.bfloat16,
    ).cuda()
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name_or_path)
datas = json.load(open('data/objaverse/cap3d_automated_objaverse_highquality_550k.json'))

/root/data/anaconda3/envs/pixellm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [2]:
new_datas = []
for data in tqdm(datas):
    caption = data['conversations'][1]['value']
    # prefix = 'Please give instructions to change the object and not remove it '
    # prompt = get_input_prompt(prefix + caption)
    prompt = get_input_prompt(caption)
    inputs = tokenizer(prompt, return_tensors='pt')
    for key, value in inputs.items():
        inputs[key] = value.cuda()
    output = model.generate(**inputs)
    instruct = tokenizer.decode(output[0, inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    data['instruct'] = instruct
    new_datas.append(data)

  0%|          | 1714/493857 [05:01<24:03:15,  5.68it/s] 


KeyboardInterrupt: 

In [3]:
new_datas[:5]

[{'id': '53d0b31aa7f84bc4b1733224963d0114',
  'image': 'objaverse/Cap3D_imgs_view0/53d0b31aa7f84bc4b1733224963d0114',
  'conversations': [{'from': 'human',
    'value': '<triplane>\nDescribe the 3D model concisely.'},
   {'from': 'gpt',
    'value': '3D rendering of a white sofa with wooden legs and frame.'}],
  'instruct': 'make the sofa a leather sofa'},
 {'id': '25f25e35aada40e49194657fd51c1199',
  'image': 'objaverse/Cap3D_imgs_view0/25f25e35aada40e49194657fd51c1199',
  'conversations': [{'from': 'human',
    'value': '<triplane>\nPresent a compact description of the 3D model key features.'},
   {'from': 'gpt',
    'value': '3D model of a wooden fence with posts and gate, featuring a sand base.'}],
  'instruct': 'make the fence made of metal'},
 {'id': '9ff331513cea44a2938d09a03d7b0493',
  'image': 'objaverse/Cap3D_imgs_view0/9ff331513cea44a2938d09a03d7b0493',
  'conversations': [{'from': 'human',
    'value': '<triplane>\nCreate a compact narrative representing the 3D model presen

In [ ]:
with open('data/objaverse/cap3d_automated_objaverse_highquality_550k.json')